# Modelling

This notebook performs model-agnostic and model-specific feature selection and modelling on the preprocessed datasets:

- Initial filters to ensure stable variables
- ElasticNet-regularized logistic regression to shrink and select sparse linear predictors
- XGBoost with BorutaShap to retain high-signal features using Shap-based importance

# Feature Selection

To begin, let's assume the goal of this project is to build an initial machine learning model for a company that, up until now, has relied on alternative methods to decide whom to lend money to and has managed to remain profitable. In this scenario, it is unlikely that the company has robust infrastructure in place for frequent model retraining, effective monitoring, or other advanced MLOps practices.

Therefore, it is critical at this stage to filter features not only for predictive power but also for their suitability in a production environment with limited support. In particular, we should prioritize variables that are stable both across different population segments (population stability) and over time (temporal stability). A population-stable feature should behave similarly when sampled from different random groups, while a temporally stable feature should maintain consistent behavior as data evolves over different time periods (e.g., maintaining its distribution, scale, and relationship with the target).

Although some unstable variables might offer additional predictive performance, initial models — especially in traditional business settings — need to focus on reliability and trust. Restricting the model to stable features lays the groundwork for sustainable deployment. More advanced versions could, in the future, incorporate riskier or less stable features once the necessary monitoring and model maintenance tools are in place to manage and mitigate any associated risks.

In [1]:
# Standard library
import json
import time

# Data manipulation
import numpy as np
import pandas as pd

# Visualization and progress bars
from tqdm import tqdm

# Graph analysis
import networkx as nx

# Feature selection utilities
from feature_engine.selection import DropConstantFeatures, DropHighPSIFeatures

# Machine learning
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
import xgboost as xgb

# Boruta SHAP for feature selection
from BorutaShap import BorutaShap

# Statistical analysis
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

# Jupyter notebook utilities
from IPython.display import clear_output

In [2]:
with open("../data/processed_model_features.json") as f:
    feature_names = json.load(f)

In [3]:
lr_data = pd.read_parquet("../data/lr_processed_data.parquet")

train_lr_data = lr_data[lr_data["dataset"] == "train"]
validation_lr_data = lr_data[lr_data["dataset"] == "validation"]

del lr_data

Now that the data is already split into training and validation/test sets, it's important to ensure that, from this point forward, we do not accidentally use information from outside the training data — such as during feature selection or hyperparameter tuning. To help enforce this and prevent data leakage, I will implement a decorator function that makes it easier to check that we are only working with the training set whenever required. If we accidentally use a DataFrame that contains any example with dataset != 'train', we will get an error.

In [4]:
def train_data_checker(func):
    """
    Decorator to check that the DataFrame passed as the first argument contains only rows with dataset == 'train'.
    Raises a ValueError if any rows with a different value are found.
    """

    def wrapper(*args, **kwargs):
        if len(args) > 0 and isinstance(args[0], pd.DataFrame):
            df = args[0]
            if "dataset" in df.columns:
                if not (df["dataset"] == "train").all():
                    raise ValueError("DataFrame contains rows where dataset != 'train'")
        return func(*args, **kwargs)

    return wrapper

### Constant

In this step, we remove near-constant features, i.e., those where the predominant value occurs in more than 99.9% of the training set. Features with so little variability are unlikely to contribute meaningful information to the model and may even hinder performance by introducing noise or unnecessary complexity.

In [5]:
@train_data_checker
def drop_constant_features(train_data, feature_names, tol=0.999, missing_values="raise", verbose=True):
    """
    Remove constant features using DropConstantFeatures from feature_engine.

    Parameters
    ----------
    train_data : pd.DataFrame
        Training data containing the features.
    feature_names : list of str
        List of feature names to check for constant values.
    tol : float, default=0.999
        The threshold below which variance is considered zero (features will be dropped).
    missing_values : str, default="raise"
        How to handle missing values.
    verbose : bool, default=True
        If True, prints diagnostics.

    Returns
    -------
    feature_names_nonconst : list of str
        Features remaining after constant feature filtering.
    dropped_features : list of str
        Features identified as constant.
    selector : DropConstantFeatures
        The fitted DropConstantFeatures object.
    """

    selector = DropConstantFeatures(tol=tol, variables=feature_names, missing_values=missing_values)
    selector.fit(train_data[feature_names])
    dropped_features = selector.features_to_drop_
    non_constant_features = [f for f in feature_names if f not in dropped_features]
    if verbose:
        print("Constant features dropped:", dropped_features)
        print("Features before constant filtering:", len(feature_names))
        print("Features after constant filtering:", len(non_constant_features))
    return non_constant_features, dropped_features, selector


non_constant_features, dropped_features, selector = drop_constant_features(train_lr_data, feature_names)
display(
    pd.Series(
        {feature: train_lr_data[feature].value_counts(normalize=True).iloc[0] for feature in dropped_features}
    )
    .sort_values(ascending=False)
    .head(10)
)

Constant features dropped: ['home_ownership_status_infrequent_sklearn', 'painter', 'pilot', 'stocker', 'business_loan', 'card_refi', 'cc_consolidation', 'citi', 'engagement', 'engagement_ring', 'motorcycle', 'pay_bills', 'pool', 'pool_loan', 'refinance_loan', 'restaurant', 'ring', 'small_business', 'accounts_120days_past_due']
Features before constant filtering: 196
Features after constant filtering: 177


restaurant         0.999979
engagement_ring    0.999850
citi               0.999829
pool_loan          0.999809
engagement         0.999796
ring               0.999738
motorcycle         0.999673
small_business     0.999540
stocker            0.999540
pay_bills          0.999500
dtype: float64

### Correlation

In the exploration notebook (01_exploration.ipynb), we briefly looked at feature correlations, but did not take corrective actions at that stage. After transforming the textual variables, it is likely that new correlated features have appeared. We will address this now.

In [6]:
@train_data_checker
def get_top_correlation_pairs(data, feature_names, sample_size=10_000, corr_method="kendall"):
    """
    Computes the top absolute pairwise correlations among the given features in data.

    Parameters
    ----------
    data : pd.DataFrame
        The input dataframe containing the features.
    feature_names : list
        List of features to compute correlations on.
    sample_size : int, optional
        If the number of rows exceeds this, a random sample is used.
    corr_method : str, optional
        Method for correlation, e.g., "kendall", "pearson", etc.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns ["feature_1", "feature_2", "absolute_correlation"], sorted descending by absolute_correlation.
    """

    correlation_sample = data[feature_names].dropna()
    if len(correlation_sample) > sample_size:
        correlation_sample = correlation_sample.sample(n=sample_size, random_state=34)

    correlation_matrix = correlation_sample.corr(method=corr_method)
    upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    top_correlation_pairs = (
        upper.stack()
        .abs()
        .sort_values(ascending=False)
        .reset_index()
        .rename(columns={"level_0": "feature_1", "level_1": "feature_2", 0: "absolute_correlation"})
    )

    return top_correlation_pairs


top_correlation_pairs = get_top_correlation_pairs(train_lr_data, non_constant_features)
top_correlation_pairs.head(15)

,feature_1,feature_2,absolute_correlation
0,fico_score_low,fico_score_high,1.000000
1,vice,vice_president,1.000000
2,buying,home_buying,1.000000
3,card_refinancing,refinancing,0.998505
4,card,credit_card,0.997664
5,home_improvement,improvement,0.997008
6,moving_relocation,relocation,0.992528
7,car_financing,financing,0.989545
8,credit,credit_card,0.979812
9,revolving_trades_with_balance,active_revolving_trades,0.977853


When reviewing the correlations, we notice that some variables appear multiple times and are often closely related to the same underlying concept (for example, `loan_purpose_home_improvement`, `home`, `improvement`, `home_improvement`, etc). To address this, we will cluster such groups of features and, from each cluster, choose a representative feature based on its correlation with the target variable.

In [7]:
def get_high_correlation_clusters(top_correlation_pairs, threshold=0.8):
    """
    Groups features into clusters where each cluster contains highly correlated features (absolute_correlation > threshold).

    Parameters
    ----------
    top_correlation_pairs : pd.DataFrame
        DataFrame with columns ["feature_1", "feature_2", "absolute_correlation"].
    threshold : float, optional
        Absolute correlation threshold for clustering.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns ["features"], each row is a cluster of highly correlated features.
    """
    high_correlation_pairs = top_correlation_pairs.query("absolute_correlation > @threshold")
    graph = nx.Graph()
    graph.add_edges_from(
        high_correlation_pairs[["feature_1", "feature_2"]].itertuples(index=False, name=None)
    )
    feature_clusters = [list(cluster) for cluster in nx.connected_components(graph)]
    clusters = pd.DataFrame(
        {
            "features": feature_clusters,
        }
    )
    return clusters


clusters = get_high_correlation_clusters(top_correlation_pairs, threshold=0.8)
clusters

,features
0,"[fico_score_high, fico_score_low]"
1,"[vice, vice_president]"
2,"[buying, home_buying]"
3,"[refinancing, card_refinancing, loan_purpose_c..."
4,"[improvement, home, loan_purpose_home_improvem..."
5,"[relocation, moving, moving_relocation]"
6,"[financing, car_financing, car, loan_purpose_car]"
7,"[revolving_trades_with_balance, active_revolvi..."
8,"[loan_purpose_debt_consolidation, debt, consol..."
9,"[medical, expenses, loan_purpose_medical, medi..."


In [8]:
@train_data_checker
def enrich_clusters_with_target_correlation(clusters, data, target_column, correlation_method="kendall"):
    """
    Enriches the clusters DataFrame with the correlation of each feature in the cluster with the target,
    highlights the selected feature (with the maximum absolute correlation to target), and the features to be removed.

    Parameters
    ----------
    clusters : pd.DataFrame
        DataFrame with a "features" column (a list of feature names).
    data : pd.DataFrame
        Data containing the features and the target variable.
    target_column : str
        Name of the target column.
    correlation_method : str, optional
        Correlation method to use (default "kendall").

    Returns
    -------
    pd.DataFrame
        DataFrame with columns:
        - features (list of features)
        - feature_target_corrs (dict: feature -> correlation with target)
        - feature_target_corr_text (str: "variable: correlation" per line)
        - selected_feature (feature with highest absolute correlation)
        - removed_features (all others)
    """
    records = []
    for _, row in clusters.iterrows():
        features = row["features"]
        correlations = {}
        # Compute correlations for features in the cluster
        for feat in features:
            if feat in data.columns:
                correlation = data[[feat, target_column]].dropna().corr(method=correlation_method).iloc[0, 1]
                correlations[feat] = f"{correlation:.2f}" if correlation is not None else "nan"

        # Order correlations first by absolute correlation (descending), then by feature name (ascending)
        ordered_corrs = sorted(
            correlations.items(),
            key=lambda kv: (-(abs(float(kv[1])) if kv[1] != "nan" else float("-inf")), kv[0]),
        )
        correlations = dict(ordered_corrs)

        if correlations:
            correlations_float = {
                k: float(v) if v != "nan" else float("-inf") for k, v in correlations.items()
            }
            selected_feature = max(correlations_float, key=lambda k: abs(correlations_float[k]))
            removed_features = [f for f in correlations if f != selected_feature]
        else:
            selected_feature = None
            removed_features = features

        records.append(
            {
                "features": features,
                "feature_target_correlation": correlations,
                "selected_feature": selected_feature,
                "removed_features": removed_features,
            }
        )

    return pd.DataFrame(records)


clusters_enriched = enrich_clusters_with_target_correlation(
    clusters, train_lr_data, target_column="default_binary", correlation_method="kendall"
)
clusters_enriched

,features,feature_target_correlation,selected_feature,removed_features
0,"[fico_score_high, fico_score_low]","{'fico_score_high': '-0.10', 'fico_score_low':...",fico_score_high,[fico_score_low]
1,"[vice, vice_president]","{'vice': '-0.01', 'vice_president': '-0.01'}",vice,[vice_president]
2,"[buying, home_buying]","{'buying': '0.01', 'home_buying': '0.01'}",buying,[home_buying]
3,"[refinancing, card_refinancing, loan_purpose_c...","{'loan_purpose_credit_card': '-0.04', 'card': ...",loan_purpose_credit_card,"[card, card_refinancing, credit, credit_card, ..."
4,"[improvement, home, loan_purpose_home_improvem...","{'home': '-0.01', 'home_improvement': '-0.01',...",home,"[home_improvement, improvement, loan_purpose_h..."
5,"[relocation, moving, moving_relocation]","{'moving': '0.01', 'moving_relocation': '0.01'...",moving,"[moving_relocation, relocation]"
6,"[financing, car_financing, car, loan_purpose_car]","{'car': '-0.01', 'car_financing': '-0.01', 'fi...",car,"[car_financing, financing, loan_purpose_car]"
7,"[revolving_trades_with_balance, active_revolvi...","{'active_revolving_trades': '0.04', 'revolving...",active_revolving_trades,[revolving_trades_with_balance]
8,"[loan_purpose_debt_consolidation, debt, consol...","{'debt_consolidation': '0.03', 'consolidation'...",debt_consolidation,"[consolidation, debt, loan_purpose_debt_consol..."
9,"[medical, expenses, loan_purpose_medical, medi...","{'expenses': '0.01', 'loan_purpose_medical': '...",expenses,"[loan_purpose_medical, medical, medical_expenses]"


In [9]:
non_correlated_features = [
    feature
    for feature in non_constant_features
    if feature not in sum(clusters_enriched["removed_features"].tolist(), [])
]
print(len(non_constant_features), len(non_correlated_features))

177 146


### Population Stability

In [10]:
@train_data_checker
def get_population_stable_features(
    train_data, feature_names, bins=10, threshold=0.25, missing_values="ignore", verbose=True
):
    """
    Detects features with high Population Stability Index (PSI) drift in train data and returns
    a filtered list of stable feature names.

    Parameters
    ----------
    train_data : pd.DataFrame
        Training data containing the features.
    feature_names : list of str
        List of feature names to check.
    bins : int, default=10
        Number of bins to use for PSI calculation.
    threshold : float, default=0.25
        PSI threshold above which features are dropped.
    missing_values : str, default="ignore"
        Whether to ignore missing values or handle them.
    verbose : bool, default=True
        If True, prints diagnostics.

    Returns
    -------
    population_stable_features : list of str
        Features remaining after PSI filtering.
    dropped_features : list of str
        Features identified as unstable by PSI.
    drop_psi_estimator : DropHighPSIFeatures
        The fitted DropHighPSIFeatures object.
    """

    drop_psi = DropHighPSIFeatures(bins=bins, threshold=threshold, missing_values=missing_values)
    drop_psi.fit(train_data[feature_names])

    drifted_features = drop_psi.features_to_drop_
    population_stable_features = [f for f in feature_names if f not in drifted_features]
    if verbose:
        print("Features with high PSI drift:", drifted_features)
        print("Features before PSI filtering:", len(feature_names))
        print("Features after PSI filtering:", len(population_stable_features))

    return population_stable_features, drifted_features, drop_psi


population_stable_features, drifted_features, drop_psi = get_population_stable_features(
    train_lr_data, non_correlated_features
)
display(pd.Series(drop_psi.psi_values_).sort_values(ascending=False).head(10))

Features with high PSI drift: []
Features before PSI filtering: 146
Features after PSI filtering: 146


months_since_oldest_revolving      0.166305
percent_trades_never_delinquent    0.160886
revolving_accounts_count           0.142899
months_since_recent_revolving      0.134823
average_current_balance            0.111372
bankcard_accounts_count            0.110502
months_since_oldest_installment    0.109317
total_installment_credit_limit     0.108483
total_high_credit_limit            0.101471
bankcard_utilization               0.096442
dtype: float64

Some features show notable PSI, but still at low or moderate levels, so we will keep them.

### Time Stability

In [11]:
def select_time_stable_features(
    data, features, sample_size=10_000, date_column="loan_issue_date", verbose=True
):
    """
    Select features that are (reasonably) stationary over time ("time-stable")
    using the Augmented Dickey-Fuller (ADF) test.

    Parameters
    ----------
    data : pd.DataFrame
        The input DataFrame containing the features and a date column.
    features : list of str
        List of feature names to check for time stability.
    sample_size : int, default=10_000
        The number of rows to sample from the data to speed up the computation.
    date_column : str, default="loan_issue_date"
        The column name containing the dates used to sort the data temporally.
    verbose : bool, default=True
        If True, prints diagnostics about feature stationarity.

    Returns
    -------
    stable_features : list of str
        Features classified as stationary (p-value < 0.05).
    nonstationary_features : list of str
        Features classified as non-stationary (p-value >= 0.05).
    adf_results : dict
        Mapping of feature name to its ADF test p-value.
    """
    time_sorted = data.sample(sample_size, random_state=34).sort_values(date_column)
    stable_features = []
    nonstationary_features = []
    adf_results = {}

    for feature in features:
        series = time_sorted[feature].values
        # Remove nans to avoid ADF errors
        series = series[~pd.isnull(series)]
        if len(series) < 10:  # skip if too short
            continue
        try:
            adf_result = adfuller(series, autolag="AIC")
            pvalue = adf_result[1]
            adf_results[feature] = pvalue
            if pvalue < 0.1:
                stable_features.append(feature)
            else:
                nonstationary_features.append(feature)
        except Exception as e:
            # Just skip features that error out
            if verbose:
                print(f"ADF failed for {feature}: {e}")
            continue
    if verbose:
        print("Time stable features:", stable_features)
        print("Time non-stable features:", nonstationary_features)

    return stable_features, nonstationary_features, adf_results


time_stable_features, time_nonstable_features, adf_results = select_time_stable_features(
    train_lr_data, population_stable_features, date_column="loan_issue_date"
)
display(pd.Series(adf_results).sort_values(ascending=False).head(10))

Time stable features: ['home_ownership_status_MORTGAGE', 'home_ownership_status_OWN', 'state_AL', 'state_AZ', 'state_CA', 'state_CO', 'state_CT', 'state_FL', 'state_GA', 'state_IL', 'state_IN', 'state_LA', 'state_MA', 'state_MD', 'state_MI', 'state_MN', 'state_MO', 'state_NC', 'state_NJ', 'state_NV', 'state_NY', 'state_OH', 'state_OR', 'state_PA', 'state_SC', 'state_TN', 'state_TX', 'state_VA', 'state_WA', 'state_WI', 'state_infrequent_sklearn', 'income_verification_status_Not Verified', 'income_verification_status_Source Verified', 'income_verification_status_Verified', 'loan_purpose_credit_card', 'loan_purpose_major_purchase', 'loan_purpose_other', 'loan_purpose_infrequent_sklearn', 'accountant', 'administrator', 'analyst', 'architect', 'attorney', 'bartender', 'bus', 'business_analyst', 'cashier', 'chef', 'clerk', 'cna', 'controller', 'cook', 'dealer', 'developer', 'director', 'driver', 'engineer', 'firefighter', 'operator', 'owner', 'pastor', 'physician', 'police', 'president', 'pr

loan                                       5.785529e-09
debt_consolidation                         2.014209e-13
consolidation_loan                         1.061572e-17
payoff                                     3.083373e-20
refinance                                  1.414099e-20
inquiries_last_6months                     4.019627e-21
card_refinance                             9.305046e-22
debt_to_income_ratio                       3.880561e-22
fico_score_high                            2.681444e-22
income_verification_status_Not Verified    7.578455e-23
dtype: float64

# Logistic Regression

This section describes the feature selection procedure using logistic regression with ElasticNet regularization. The goal is to identify a stable and predictive subset of features by applying a stability selection approach. Specifically, we repeatedly fit a logistic regression model with ElasticNet penalty (using stochastic gradient descent) to different subsamples of the training data. This is done across multiple cross-validation folds and bootstrap runs, controlling the strength of regularization and the relative weighting of L1 and L2 penalties.

For each run, we record which features receive non-zero coefficients. At the end of all runs, we calculate for each feature the proportion of times it was selected, and retain only those features that are consistently selected above a user-defined threshold. This process helps ensure that only robust and non-redundant features are kept, improving generalizability and model interpretability.

In [12]:
stable_features = time_stable_features
print(stable_features)
target = "default_binary"

X_train_lr = train_lr_data[stable_features].to_numpy()
X_validation_lr = validation_lr_data[stable_features].to_numpy()

y_train_lr = train_lr_data[target]
y_validation_lr = validation_lr_data[target]

del train_lr_data, validation_lr_data

['home_ownership_status_MORTGAGE', 'home_ownership_status_OWN', 'state_AL', 'state_AZ', 'state_CA', 'state_CO', 'state_CT', 'state_FL', 'state_GA', 'state_IL', 'state_IN', 'state_LA', 'state_MA', 'state_MD', 'state_MI', 'state_MN', 'state_MO', 'state_NC', 'state_NJ', 'state_NV', 'state_NY', 'state_OH', 'state_OR', 'state_PA', 'state_SC', 'state_TN', 'state_TX', 'state_VA', 'state_WA', 'state_WI', 'state_infrequent_sklearn', 'income_verification_status_Not Verified', 'income_verification_status_Source Verified', 'income_verification_status_Verified', 'loan_purpose_credit_card', 'loan_purpose_major_purchase', 'loan_purpose_other', 'loan_purpose_infrequent_sklearn', 'accountant', 'administrator', 'analyst', 'architect', 'attorney', 'bartender', 'bus', 'business_analyst', 'cashier', 'chef', 'clerk', 'cna', 'controller', 'cook', 'dealer', 'developer', 'director', 'driver', 'engineer', 'firefighter', 'operator', 'owner', 'pastor', 'physician', 'police', 'president', 'product', 'professor', '

In [13]:
def feature_selection_lr(
    X,
    y,
    n_splits=3,
    n_boot=10,
    subsample=0.7,
    alpha=1e-5,
    l1_ratio=0.5,
    threshold=0.75,
    max_iter=5,
    tol=1e-3,
    random_state=34,
    class_weight="balanced",
):
    """
    Efficient stability selection for feature selection using ElasticNet logistic regression via SGD.

    Selects features that are nonzero in at least `threshold` fraction of runs (cross-validation × bootstraps).

    Returns
    -------
    selected_idx : np.ndarray
        Indices of features selected.
    freq : np.ndarray
        Fraction of times each feature was selected (between 0 and 1).
    """
    if hasattr(y, "to_numpy"):
        y = y.to_numpy().ravel()
    else:
        y = np.asarray(y).ravel()

    n_features = X.shape[1]
    counts = np.zeros(n_features, dtype=np.int32)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    rng = np.random.default_rng(random_state)

    total_runs = n_splits * n_boot
    progress_bar = tqdm(total=total_runs, desc="LR Feature Selection", leave=True)

    run = 0
    for fold_index, (tr_idx, _) in enumerate(skf.split(np.zeros_like(y), y), start=1):
        X_fold = X[tr_idx]
        y_fold = y[tr_idx]
        n_fold = len(tr_idx)

        for _ in range(n_boot):
            run += 1
            m = int(subsample * n_fold)
            sub_local = rng.choice(n_fold, size=m, replace=False)

            X_sub = X_fold[sub_local]
            y_sub = y_fold[sub_local]

            classifier = SGDClassifier(
                loss="log_loss",
                penalty="elasticnet",
                alpha=alpha,
                l1_ratio=l1_ratio,
                max_iter=max_iter,
                tol=tol,
                class_weight=class_weight,
                random_state=random_state + 1000 * fold_index + run,
            )
            classifier.fit(X_sub, y_sub)

            counts += classifier.coef_.ravel() != 0

            progress_bar.update(1)
            progress_bar.set_description(f"Fold {fold_index}/{n_splits}")

    progress_bar.close()

    frequency = counts / float(total_runs)
    selected_indices = np.flatnonzero(frequency >= threshold)
    return selected_indices, frequency


# we are only using the train data for feature selection
selected_indices, frequency = feature_selection_lr(
    X_train_lr,
    y_train_lr,
    n_splits=3,  # 3 is usually enough and faster than 5
    n_boot=10,  # start with 10; increase to 20 if selection is too noisy
    subsample=0.7,
    alpha=3e-4,  # increase (e.g., 3e-4) to select fewer features; decrease to select more
    l1_ratio=0.85,
    threshold=0.9,  # increase to be stricter (fewer features), decrease to include more
    max_iter=5,
    tol=1e-3,
    random_state=34,
)

print("Selected features:", selected_indices.size, "out of", X_train_lr.shape[1])
selected_features_lr = [stable_features[i] for i in selected_indices]
print(selected_features_lr)

Fold 3/3: 100%|██████████| 30/30 [01:00<00:00,  2.01s/it]           

Selected features: 39 out of 146
['home_ownership_status_MORTGAGE', 'home_ownership_status_OWN', 'state_CO', 'state_FL', 'state_GA', 'state_IL', 'state_NV', 'state_NY', 'state_OR', 'state_SC', 'state_WA', 'income_verification_status_Not Verified', 'income_verification_status_Verified', 'loan_purpose_credit_card', 'loan_purpose_other', 'loan_purpose_infrequent_sklearn', 'director', 'driver', 'engineer', 'owner', 'sales', 'business', 'annual_income', 'loan_amount_requested', 'fico_score_high', 'total_credit_lines', 'revolving_balance', 'active_revolving_trades', 'months_since_oldest_revolving', 'total_installment_credit_limit', 'bankcard_open_to_buy', 'months_since_recent_bankcard', 'percent_bankcard_over_75pct_limit', 'inquiries_last_6months', 'months_since_recent_inquiry', 'accounts_opened_past_12months', 'accounts_opened_past_24months', 'total_high_credit_limit', 'total_bankcard_limit']


By applying this feature selection method, we were able to reduce the number of features from almost 200 to approximately 40. This substantial reduction makes the model lighter and more efficient, since it focuses only on the most relevant predictors. 

This helps us in several ways:
- The model is less likely to overfit, which means we can expect it to generalize better to new, unseen data and to behave more consistently.
- With fewer features, the model becomes easier to interpret, allowing us to better understand the drivers of predictions and communicate results to stakeholders.
- A more compact feature set simplifies initial model monitoring after deployment, making it easier to identify issues, track changes, and validate assumptions in production.

Overall, careful feature selection increases our confidence in building a robust, reliable, and maintainable model.

In [14]:
lr_data = pd.read_parquet("../data/lr_processed_data.parquet")

lr_features_renamed = {f: f"lr_{f}" for f in selected_features_lr}
print(list(lr_features_renamed.values()))

lr_data = lr_data.rename(columns=lr_features_renamed)
X_train_df = lr_data[lr_data["dataset"] == "train"][list(lr_features_renamed.values())]
X_validation_df = lr_data[lr_data["dataset"] == "validation"][list(lr_features_renamed.values())]
y_train = lr_data[lr_data["dataset"] == "train"]["default_binary"]
y_validation = lr_data[lr_data["dataset"] == "validation"]["default_binary"]

del lr_data

X_train = sm.add_constant(X_train_df[list(lr_features_renamed.values())])
X_validation = sm.add_constant(X_validation_df[list(lr_features_renamed.values())])

start_time = time.time()
logit_model = sm.Logit(y_train_lr, X_train)
lr_model = logit_model.fit()
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

validation_predicted = lr_model.predict(X_validation)
validation_auc = roc_auc_score(y_validation_lr, validation_predicted)
print(f"\nValidation AUC: {validation_auc:.4f}")

['lr_home_ownership_status_MORTGAGE', 'lr_home_ownership_status_OWN', 'lr_state_CO', 'lr_state_FL', 'lr_state_GA', 'lr_state_IL', 'lr_state_NV', 'lr_state_NY', 'lr_state_OR', 'lr_state_SC', 'lr_state_WA', 'lr_income_verification_status_Not Verified', 'lr_income_verification_status_Verified', 'lr_loan_purpose_credit_card', 'lr_loan_purpose_other', 'lr_loan_purpose_infrequent_sklearn', 'lr_director', 'lr_driver', 'lr_engineer', 'lr_owner', 'lr_sales', 'lr_business', 'lr_annual_income', 'lr_loan_amount_requested', 'lr_fico_score_high', 'lr_total_credit_lines', 'lr_revolving_balance', 'lr_active_revolving_trades', 'lr_months_since_oldest_revolving', 'lr_total_installment_credit_limit', 'lr_bankcard_open_to_buy', 'lr_months_since_recent_bankcard', 'lr_percent_bankcard_over_75pct_limit', 'lr_inquiries_last_6months', 'lr_months_since_recent_inquiry', 'lr_accounts_opened_past_12months', 'lr_accounts_opened_past_24months', 'lr_total_high_credit_limit', 'lr_total_bankcard_limit']
Optimization te

At this point, you might be wondering: "Why is the AUC value comparatively low?" The reason is that different problems with varying levels of complexity have different expected results. While with simpler datasets we could easily achieve metrics above 0.9, in this case, the current value is already considered reasonable given the problem's difficulty.

It is important to note that AUC itself is not the ultimate metric here, but it does provide a good indication of how well the model ranks predictions. This property is particularly valuable, as it translates into actionable insights for the business: if a model can rank well, it empowers us to work with different score bands to make more informed lending decisions, such as how much to lend, to whom, and under what conditions. This makes the model's output not just a number, but a practical guide for real-world decisions.

# XGBoost

In [15]:
xgb_data = pd.read_parquet("../data/xgb_processed_data.parquet")

train_xgb_data = xgb_data[xgb_data["dataset"] == "train"]
validation_xgb_data = xgb_data[xgb_data["dataset"] == "validation"]

del xgb_data

In [16]:
non_constant_features, dropped_features, selector = drop_constant_features(
    train_xgb_data, feature_names, missing_values="include"
)

Constant features dropped: ['home_ownership_status_infrequent_sklearn', 'painter', 'pilot', 'stocker', 'business_loan', 'card_refi', 'cc_consolidation', 'citi', 'engagement', 'engagement_ring', 'motorcycle', 'pay_bills', 'pool', 'pool_loan', 'refinance_loan', 'restaurant', 'ring', 'small_business']
Features before constant filtering: 196
Features after constant filtering: 178


In [17]:
top_correlation_pairs = get_top_correlation_pairs(train_xgb_data, non_constant_features)
clusters = get_high_correlation_clusters(top_correlation_pairs, threshold=0.8)
clusters_enriched = enrich_clusters_with_target_correlation(
    clusters, train_xgb_data, target_column="default_binary", correlation_method="kendall"
)
non_correlated_features = [
    feature
    for feature in non_constant_features
    if feature not in sum(clusters_enriched["removed_features"].tolist(), [])
]
print(len(non_constant_features), len(non_correlated_features))

178 147


In [18]:
population_stable_features, drifted_features, drop_psi = get_population_stable_features(
    train_xgb_data, non_correlated_features
)

Features with high PSI drift: []
Features before PSI filtering: 147
Features after PSI filtering: 147


In [19]:
time_stable_features, time_nonstable_features, adf_results = select_time_stable_features(
    train_xgb_data, population_stable_features, date_column="loan_issue_date"
)

Time stable features: ['home_ownership_status_MORTGAGE', 'home_ownership_status_OWN', 'state_AL', 'state_AZ', 'state_CA', 'state_CO', 'state_CT', 'state_FL', 'state_GA', 'state_IL', 'state_IN', 'state_LA', 'state_MA', 'state_MD', 'state_MI', 'state_MN', 'state_MO', 'state_NC', 'state_NJ', 'state_NV', 'state_NY', 'state_OH', 'state_OR', 'state_PA', 'state_SC', 'state_TN', 'state_TX', 'state_VA', 'state_WA', 'state_WI', 'state_infrequent_sklearn', 'income_verification_status_Not Verified', 'income_verification_status_Source Verified', 'income_verification_status_Verified', 'loan_purpose_credit_card', 'loan_purpose_major_purchase', 'loan_purpose_other', 'loan_purpose_infrequent_sklearn', 'accountant', 'administrator', 'analyst', 'architect', 'attorney', 'bartender', 'bus', 'business_analyst', 'cashier', 'chef', 'clerk', 'cna', 'controller', 'cook', 'dealer', 'developer', 'director', 'driver', 'engineer', 'firefighter', 'operator', 'owner', 'pastor', 'physician', 'police', 'president', 'pr

In [20]:
stable_features = time_stable_features
print(stable_features)
target = "default_binary"

X_train_xgb = train_xgb_data[stable_features].to_numpy()
X_validation_xgb = validation_xgb_data[stable_features].to_numpy()

y_train_xgb = train_xgb_data[target]
y_validation_xgb = validation_xgb_data[target]

del train_xgb_data, validation_xgb_data

['home_ownership_status_MORTGAGE', 'home_ownership_status_OWN', 'state_AL', 'state_AZ', 'state_CA', 'state_CO', 'state_CT', 'state_FL', 'state_GA', 'state_IL', 'state_IN', 'state_LA', 'state_MA', 'state_MD', 'state_MI', 'state_MN', 'state_MO', 'state_NC', 'state_NJ', 'state_NV', 'state_NY', 'state_OH', 'state_OR', 'state_PA', 'state_SC', 'state_TN', 'state_TX', 'state_VA', 'state_WA', 'state_WI', 'state_infrequent_sklearn', 'income_verification_status_Not Verified', 'income_verification_status_Source Verified', 'income_verification_status_Verified', 'loan_purpose_credit_card', 'loan_purpose_major_purchase', 'loan_purpose_other', 'loan_purpose_infrequent_sklearn', 'accountant', 'administrator', 'analyst', 'architect', 'attorney', 'bartender', 'bus', 'business_analyst', 'cashier', 'chef', 'clerk', 'cna', 'controller', 'cook', 'dealer', 'developer', 'director', 'driver', 'engineer', 'firefighter', 'operator', 'owner', 'pastor', 'physician', 'police', 'president', 'product', 'professor', '

In [21]:
def feature_selection_xgb(
    X,
    y,
    feature_names,
    n_boot=10,
    subsample=0.7,
    threshold=0.7,
    boruta_percentile=80,
    boruta_pvalue=0.1,
    random_state=34,
    xgb_params=None,
    n_trials=30,
    sample_size=10_000,
):
    """
    Stability selection for feature selection using BorutaShap with bootstrap strategy.

    For each bootstrap, runs BorutaShap and counts inclusion frequency per feature.
    Selects features kept in at least `threshold` fraction of runs.

    Returns
    -------
    selected_features: list
        Features selected by stability selection with BorutaShap.
    frequency: pd.Series
        Fraction of times each feature was selected (indexed by feature name).
    """
    if hasattr(y, "to_numpy"):
        y = y.to_numpy().ravel()
    else:
        y = np.asarray(y).ravel()

    if X.shape[0] > sample_size:
        sss = StratifiedShuffleSplit(n_splits=1, train_size=sample_size, random_state=random_state)
        idx, _ = next(sss.split(X, y))
        X = X[idx]
        y = y[idx]

    n_features = X.shape[1]
    counts = np.zeros(n_features, dtype=np.int32)
    rng = np.random.default_rng(random_state)
    total_runs = n_boot
    progress_bar = tqdm(total=total_runs, desc="BorutaShap Stability Selection", leave=True)

    if xgb_params is None:
        xgb_estimator = xgb.XGBClassifier(
            n_estimators=400,
            max_depth=6,
            learning_rate=0.08,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_lambda=1.0,
            reg_alpha=0.0,
            eval_metric="auc",
            device="cuda",
            n_jobs=-1,
            random_state=random_state,
        )
    else:
        xgb_estimator = xgb.XGBClassifier(**xgb_params)

    n_samples = X.shape[0]

    for boot in range(n_boot):
        m = int(subsample * n_samples)
        sub_local = rng.choice(n_samples, size=m, replace=False)
        X_sub = X[sub_local]
        y_sub = y[sub_local]

        # BorutaShap expects pandas DataFrame
        if hasattr(X_sub, "toarray"):
            X_df = pd.DataFrame(X_sub.toarray(), columns=feature_names)
        else:
            X_df = pd.DataFrame(X_sub, columns=feature_names)

        boruta_selector = BorutaShap(
            model=xgb_estimator,
            importance_measure="shap",
            classification=True,
            percentile=boruta_percentile,
            pvalue=boruta_pvalue,
        )

        boruta_selector.fit(
            X=X_df,
            y=y_sub,
            sample=False,
            train_or_test="train",
            n_trials=n_trials,
            random_state=random_state + boot,
        )

        boruta_selector.TentativeRoughFix()

        features_removed = boruta_selector.features_to_remove
        selected = ~np.isin(feature_names, features_removed)
        counts += selected.astype(np.int32)

        progress_bar.update(1)
        progress_bar.set_description(f"Bootstrap {boot+1}/{n_boot}")

    progress_bar.close()
    frequency = counts / float(total_runs)
    selected_features = [feature_names[i] for i, freq in enumerate(frequency) if freq >= threshold]

    clear_output()

    return selected_features, pd.Series(frequency, index=feature_names)


selected_features_xgb, frequency_borutashap = feature_selection_xgb(
    X_train_xgb,
    y_train_xgb,
    feature_names=stable_features,
)
print(selected_features_xgb)

['employment_length_years', 'annual_income', 'loan_amount_requested', 'fico_score_high', 'debt_to_income_ratio', 'total_credit_lines', 'revolving_balance', 'revolving_utilization_rate', 'revolving_accounts_count', 'months_since_oldest_revolving', 'months_since_oldest_installment', 'total_installment_credit_limit', 'satisfactory_bankcard_accounts', 'bankcard_utilization', 'bankcard_open_to_buy', 'months_since_recent_bankcard', 'percent_bankcard_over_75pct_limit', 'months_since_recent_inquiry', 'accounts_opened_past_12months', 'accounts_opened_past_24months', 'total_high_credit_limit', 'total_balance_excluding_mortgage', 'total_bankcard_limit', 'average_current_balance', 'mortgage_accounts_count', 'percent_trades_never_delinquent']


In [22]:
xgb_data = pd.read_parquet("../data/xgb_processed_data.parquet")

xgb_features_renamed = {f: f"xgb_{f}" for f in selected_features_xgb}
print(list(xgb_features_renamed.values()))

xgb_data = xgb_data.rename(columns=xgb_features_renamed)
X_train_df = xgb_data[xgb_data["dataset"] == "train"][list(xgb_features_renamed.values())]
X_validation_df = xgb_data[xgb_data["dataset"] == "validation"][list(xgb_features_renamed.values())]
y_train = xgb_data[xgb_data["dataset"] == "train"]["default_binary"]
y_validation = xgb_data[xgb_data["dataset"] == "validation"]["default_binary"]

del xgb_data

X_train = X_train_df[list(xgb_features_renamed.values())]
X_validation = X_validation_df[list(xgb_features_renamed.values())]

xgb_model = xgb.XGBClassifier(
    n_estimators=400,
    max_depth=6,
    learning_rate=0.08,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    reg_alpha=0.0,
    eval_metric="auc",
    n_jobs=-1,
    device="cuda",
    random_state=34,
)

start_time = time.time()
xgb_model.fit(X_train, y_train)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

validation_predicted = xgb_model.predict_proba(X_validation)[:, 1]
validation_auc = roc_auc_score(y_validation, validation_predicted)
print(f"\nValidation AUC: {validation_auc:.4f}")

['xgb_employment_length_years', 'xgb_annual_income', 'xgb_loan_amount_requested', 'xgb_fico_score_high', 'xgb_debt_to_income_ratio', 'xgb_total_credit_lines', 'xgb_revolving_balance', 'xgb_revolving_utilization_rate', 'xgb_revolving_accounts_count', 'xgb_months_since_oldest_revolving', 'xgb_months_since_oldest_installment', 'xgb_total_installment_credit_limit', 'xgb_satisfactory_bankcard_accounts', 'xgb_bankcard_utilization', 'xgb_bankcard_open_to_buy', 'xgb_months_since_recent_bankcard', 'xgb_percent_bankcard_over_75pct_limit', 'xgb_months_since_recent_inquiry', 'xgb_accounts_opened_past_12months', 'xgb_accounts_opened_past_24months', 'xgb_total_high_credit_limit', 'xgb_total_balance_excluding_mortgage', 'xgb_total_bankcard_limit', 'xgb_average_current_balance', 'xgb_mortgage_accounts_count', 'xgb_percent_trades_never_delinquent']
Training time: 5.27 seconds

Validation AUC: 0.6873


With a more competitive model like XGBoost, we achieved an AUC increase of 0.01. While this may not seem significant, it is important to note two things: no dedicated hyperparameter tuning was performed for this model, which could potentially boost AUC to around 0.7, and even this current improvement might lead to valuable results that we will explore in the next notebook.

# Persist datasets and predictions

Nós já temos as variáveis iniciais selecionadas (já filtradas para serem o mais estável possível)

In [23]:
xgb_data = pd.read_parquet("../data/xgb_processed_data.parquet")

data = xgb_data.drop(columns=feature_names)
print(data.shape)

xgb_features_renamed = {f: f"xgb_{f}" for f in selected_features_xgb}
print(list(xgb_features_renamed.values()))
data = pd.merge(
    data,
    xgb_data[["loan_id"] + selected_features_xgb].rename(columns=xgb_features_renamed),
    on="loan_id",
    how="left",
)
print(data.shape)

del xgb_data

lr_data = pd.read_parquet("../data/lr_processed_data.parquet")

lr_features_renamed = {f: f"lr_{f}" for f in selected_features_lr}
print(list(lr_features_renamed.values()))
data = pd.merge(
    data,
    lr_data[["loan_id"] + selected_features_lr].rename(columns=lr_features_renamed),
    on="loan_id",
    how="left",
)
print(data.shape)

del lr_data

data["xgb_pred"] = xgb_model.predict_proba(data[list(xgb_features_renamed.values())].to_numpy())[:, 1]
data["lr_pred"] = lr_model.predict(sm.add_constant(data[list(lr_features_renamed.values())]))

data.head()

(1059906, 96)
['xgb_employment_length_years', 'xgb_annual_income', 'xgb_loan_amount_requested', 'xgb_fico_score_high', 'xgb_debt_to_income_ratio', 'xgb_total_credit_lines', 'xgb_revolving_balance', 'xgb_revolving_utilization_rate', 'xgb_revolving_accounts_count', 'xgb_months_since_oldest_revolving', 'xgb_months_since_oldest_installment', 'xgb_total_installment_credit_limit', 'xgb_satisfactory_bankcard_accounts', 'xgb_bankcard_utilization', 'xgb_bankcard_open_to_buy', 'xgb_months_since_recent_bankcard', 'xgb_percent_bankcard_over_75pct_limit', 'xgb_months_since_recent_inquiry', 'xgb_accounts_opened_past_12months', 'xgb_accounts_opened_past_24months', 'xgb_total_high_credit_limit', 'xgb_total_balance_excluding_mortgage', 'xgb_total_bankcard_limit', 'xgb_average_current_balance', 'xgb_mortgage_accounts_count', 'xgb_percent_trades_never_delinquent']
(1059906, 122)
['lr_home_ownership_status_MORTGAGE', 'lr_home_ownership_status_OWN', 'lr_state_CO', 'lr_state_FL', 'lr_state_GA', 'lr_state_IL

,loan_id,loan_amount_funded,loan_amount_funded_investors,loan_term_months,interest_rate,monthly_payment,loan_grade,loan_subgrade,employment_title,home_ownership_status,...,lr_months_since_recent_bankcard,lr_percent_bankcard_over_75pct_limit,lr_inquiries_last_6months,lr_months_since_recent_inquiry,lr_accounts_opened_past_12months,lr_accounts_opened_past_24months,lr_total_high_credit_limit,lr_total_bankcard_limit,xgb_pred,lr_pred
0,1077501,5000.0,4975.0,36,0.1065,162.87,B,B2,<NA>,RENT,...,0.020344,0.5,0.125,0.2,0.0625,0.0625,0.711753,0.686046,0.143411,0.136394
1,1077175,2400.0,2400.0,36,0.1596,84.33,C,C5,<NA>,RENT,...,0.020344,0.5,0.250,0.2,0.0625,0.0625,0.711753,0.686046,0.193209,0.157012
2,1076863,10000.0,10000.0,36,0.1349,339.31,C,C1,AIR RESOURCES BOARD,RENT,...,0.020344,0.5,0.125,0.2,0.0625,0.0625,0.711753,0.686046,0.139291,0.183439
3,1075269,5000.0,5000.0,36,0.0790,156.46,A,A4,Veolia Transportaton,RENT,...,0.020344,0.5,0.375,0.2,0.0625,0.0625,0.711753,0.686046,0.088156,0.185711
4,1072053,3000.0,3000.0,36,0.1864,109.43,E,E1,MKC Accounting,RENT,...,0.020344,0.5,0.250,0.2,0.0625,0.0625,0.711753,0.686046,0.214369,0.184568


In [24]:
data.to_parquet("../data/scored_data.parquet")